## Review Docker File

In [78]:
!cat container/Dockerfile

FROM python:3.6

RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

# Install all of the packages
RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

# install code dependencies
COPY "requirements.txt" .
RUN ["pip", "install", "-r", "requirements.txt"]

RUN pip list
# Env Variables
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/ml:${PATH}"

# Set up the program in the image
COPY scripts /opt/ml
WORKDIR /opt/ml



# Main Libraries used:

# RUN pip install numpy
# RUN pip install scipy
# RUN pip install scikit-learn
# RUN pip install pandas
# RUN pip install flask
# RUN pip install gevent
# RUN pip install gunicorn
# RUN pip install tensorflow==1.15.2
# RUN pip install keras==2.2.4
# RUN pip install h5py
# RUN pip install hyperopt


## Building and registering the container

In [67]:
%%sh

# The name of our algorithm
algorithm_name=dnn

cd container

chmod +x scripts/train
chmod +x scripts/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/11 : FROM python:3.6
 ---> 13efce2de907
Step 2/11 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 14ce9b4922bb
Step 3/11 : RUN wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py
 ---> Using cache
 ---> bfd39c0dd3bf
Step 4/11 : COPY "requirements.txt" .
 ---> Using cache
 ---> 6f52b9af68ab
Step 5/11 : RUN ["pip", "install", "-r", "requirements.txt"]
 ---> Using cache
 ---> 6e59c4d394ac
Step 6/11 : RUN pip list
 ---> Using cache
 ---> b4643e53e98c
Step 7/11 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> f061c68f2ab9
Step 8/11 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 3c2ebefdf487
Step 9/11 : ENV PATH="/opt/ml:${PATH}"
 ---> Using cache
 ---> 9188186bdb9e
Step 10/11 : COPY scripts /opt/ml
 ---> Using cache
 ---> cdf3064ac80f
Step 11/11 : WORKDIR /opt/ml
 ---> Using cache
 ---> 348f7d

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



# Testing the container through notebook

In [68]:
# S3 prefix
prefix = 'dnn'

# Define IAM role
import boto3
import re
import io
import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role
import json
from sklearn.preprocessing import StandardScaler

role = get_execution_role()

## Create the session

In [69]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## Upload the data for training

In [70]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [71]:
data_location

's3://sagemaker-us-east-1-932202118120/garrett'

In [72]:
!aws s3 ls s3://sagemaker-us-east-1-597844091762/garrett/


An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied


## Create an estimator and fit the model

In [73]:
# !container/local_test/train_local.sh ann-churn

In [74]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{prefix}:latest'

clf = sage.estimator.Estimator(
    image, 
    role, 
    1, 
    'ml.m4.2xlarge',
    output_path="s3://{}/{}/output".format(sess.default_bucket(), prefix),
    sagemaker_session=sess)

In [76]:
###############################################
###########   Default HPO parameters  #########
###############################################

# target = 'PE_',  # arbitrary string
# batch_normalization = True,  # 
# include_dropout = True,
# dropout = [.2,.5],
# early_stopping_patience = 15, # Number of epochs with no significant change in metric before early stopping happens 
# lr_update_patience = 7, # Number of epochs with no significant change in metric before learning rate decrease
# used_data_percentage = 10,
# train_validation_split = .15,
# MAX_EVALS = 3,
# randstate = 50,
# num_layers_low = 6,
# num_layers_high = 9,
# choice_of_node_numbers = [16,32,64,128,256,512,1024,2048], # Here you can give the possible node size for layers. If you want to only have small number of nodes, remove the high values from this list. 
# nb_epochs = 5,
# batch_size = [16,32,64,128],
# optimizer = ['adam'],
# last_activation_f = ['tanh']  # Activation for the layer with one node. Options for this are 'linear' and 'tanh'


##############################################
######### Default  Final Model       #########
##############################################
# target = 'PE_',
# batch_normalization = True,
# include_dropout = True,
# dropout_f = .2,
# early_stopping_patience = 15, # Number of epochs with no significant change in metric before early stopping happens 
# lr_update_patience = 7, # Number of epochs with no significant change in metric before learning rate decrease
#num_layers_f = 8
#nodes = [1024,524,256,128,64,32,16]
#nb_epochs_f = 1
#batch_size_f = 32
#optimizer_f = 'adam'
# last_activation_f = 'tanh'


final_training = False   

if final_training: # Final Training
    clf.set_hyperparameters(final_training = True,
                            target = 'PE_',
                            batch_normalization = True,
                            include_dropout = True,
                            dropout_f = .2,
                            early_stopping_patience = 15,
                            lr_update_patience = 7,
                            loss_metric = 'mae',
                            monitor_metric = 'val_mean_absolute_error',
                            num_layers_f = 8,
                            nodes = [1024,64,1024,32,32,64,512], # The number of nodes (length of "nodes" list) should be num_layers_f-1 because the last layer has 1 node and is automatically added
                            nb_epochs_f = 3,
                            batch_size_f = 32,
                            optimizer_f = 'adam',
                            last_activation_f = 'tanh'
                           )    
else:  # HPO
    clf.set_hyperparameters(final_training = False,
                            target = 'PE_',
                            batch_normalization = True,
                            include_dropout = False,
                            dropout = [.2,.3,.5],
                            early_stopping_patience = 15,
                            lr_update_patience = 7,
                            loss_metric = 'mae',
                            monitor_metric = 'val_mean_absolute_error',
                            used_data_percentage = 10,
                            train_validation_split = .15,
                            MAX_EVALS = 3,
                            randstate = 50,
                            num_layers_low = 1,
                            num_layers_high = 9,
                            choice_of_node_numbers = [16,32,64,128,256,512,1024,2048], # Here you can give the possible node size for layers. If you want to only have small number of nodes, remove the high values from this list. 
                            nb_epochs = 3,
                            batch_size = [32,64,128],
                            optimizer = ['adam'],
                            last_activation = ['tanh']  # Activation for the layer with one node. Options for this are 'linear' and 'tanh'
                            )


In [ ]:
clf.fit(data_location)

2020-07-08 16:12:28 Starting - Starting the training job...
2020-07-08 16:12:31 Starting - Launching requested ML instances......
2020-07-08 16:13:51 Starting - Preparing the instances for training......
2020-07-08 16:14:58 Downloading - Downloading input data
2020-07-08 16:14:58 Training - Downloading the training image..

## Deploy the model

In [48]:
from sagemaker.predictor import csv_serializer

# 'ml.m4.2xlarge
predictor = clf.deploy(1, 'ml.t2.medium', serializer=csv_serializer)

----------------------!

## Inference

In [49]:
endpointName = 'endpoint-2020-06-16-15-27-13-763' # Your endpoint name that was created in "Deploy the model" section

from pickle import load

# Process and prepare the data
def transform_data(test_x):
    test_x = test_x.dropna()
    test_x = test_x.astype('float32')
        
    # Feature Scaling
    scaler = load(open('scaler.pkl', 'rb'))
    test_x = scaler.fit_transform(test_x)
    return pd.DataFrame(test_x)

test_X = pd.read_csv('SL_test_DG_mod.csv')

test_X = transform_data(test_X)


test_file = io.StringIO()
test_X.to_csv(test_file, header=None, index=None)

# Talk to SageMaker
client = boto3.client('sagemaker-runtime')
response = client.invoke_endpoint(
    EndpointName=endpointName,
    Body=test_file.getvalue(),
    ContentType='text/csv',
    Accept='Accept'
)

print(response['Body'].read().decode('ascii'))

0.74156004
0.73478514
0.74325
0.6297486
0.7326459
0.73808956
0.7359009
0.7388833
0.73829156
0.7397905
0.7380741
0.7307031
0.74140054
0.74096894
0.73406225
0.66092
0.7370605
0.74014443
0.7391198
0.73886997
0.74157727
0.7389096
0.74006116
0.74000776
0.66124535
0.6710666
0.7370405
0.7392786
0.7389998
0.74078166
0.7187387
0.7382029
0.7413171
0.7422559
0.73747265
0.73424584
0.73487234
0.7403716
0.7373593
0.7387279
0.73849475
0.7380303
0.73891
0.690313
0.7411712
0.74161386
0.7413369
0.55384463
0.73815763
0.7297554
0.7382472
-0.18515208
0.74046016
0.73924273
0.5890239
0.7419111
0.5734794
0.7406449
0.73937804
0.7403185
0.7415058
0.74109095
0.740117
0.30571777
0.7392786
0.74279076
0.7200922
0.7390448
0.14644182
0.73875475
0.7373179
0.74119896
0.7415056
0.7379794
0.7349165
0.53299654
0.7382031
0.7398333
0.7386627
0.7401146
0.73779494
0.7387914
0.73697364
0.73649675
0.7373422
0.7405447
0.7375472
0.73957497
0.74106634
0.5669162
0.736945
0.7006622
0.7416847
0.2522543
0.71346027
0.72729784
0.7380501

In [ ]:
# !tar cvfz container_hpo.tar.gz *

## Optional cleanup

In [17]:
# sess.delete_endpoint(predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: ann-churn-2018-05-27-18-29-21-010
